In [13]:
from datasets import load_dataset, disable_caching
import os

disable_caching()

In [18]:
data_files = {}
train_file = "data/train_clevr_obj.json"
validation_file = "data/dev_clevr_obj.json"
cache_dir = "cache"
if train_file is not None:
    data_files["train"] = train_file
    extension = train_file.split(".")[-1]
if validation_file is not None:
    data_files["validation"] = validation_file
    extension = validation_file.split(".")[-1]
dataset = load_dataset(extension, data_files=data_files, cache_dir=cache_dir)

Found cached dataset json (/home/khangln/DRIVE_H/WORK/llms/vision/contrastive_img_txt/cache/json/default-ab1f67df85342d82/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 2/2 [00:00<00:00, 248.76it/s]


In [19]:
img_dir = 'clevr_img'


def add_full_img_path_and_caption(example):
    example["caption"] = f"A photo of a {example['color']} {example['material']} {example['shape']}"
    example['img'] = os.path.join(img_dir, example['image'])
    return example

In [20]:
dataset = dataset.map(add_full_img_path_and_caption)


In [22]:
dataset["validation"][0]

{'color': 'green',
 'material': 'rubber',
 'shape': 'sphere',
 'image': 'img_800.png',
 'caption': 'A photo of a green rubber sphere',
 'img': 'clevr_img/img_800.png'}

In [2]:
import json 
import os
with open("data/test_unseen_shape.json") as f:
    test_data = [json.loads(line) for line in f]
img_dir = "clevr_images/unseen_shape"

for d in test_data:
    d["image_path"] = os.path.join(img_dir, d["image"])
    d["label"] = f"{d['color']} {d['material']} {d['shape']}"

In [3]:
from torch.utils.data import DataLoader

DataLoader(test_data, batch)

[{'color': 'blue',
  'material': 'rubber',
  'shape': 'donut',
  'image': 'img_0.png',
  'image_path': 'clevr_images/unseen_shape/img_0.png',
  'label': 'blue rubber donut'},
 {'color': 'red',
  'material': 'metal',
  'shape': 'cone',
  'image': 'img_1.png',
  'image_path': 'clevr_images/unseen_shape/img_1.png',
  'label': 'red metal cone'},
 {'color': 'yellow',
  'material': 'metal',
  'shape': 'donut',
  'image': 'img_2.png',
  'image_path': 'clevr_images/unseen_shape/img_2.png',
  'label': 'yellow metal donut'},
 {'color': 'green',
  'material': 'metal',
  'shape': 'cone',
  'image': 'img_3.png',
  'image_path': 'clevr_images/unseen_shape/img_3.png',
  'label': 'green metal cone'},
 {'color': 'red',
  'material': 'metal',
  'shape': 'donut',
  'image': 'img_4.png',
  'image_path': 'clevr_images/unseen_shape/img_4.png',
  'label': 'red metal donut'},
 {'color': 'blue',
  'material': 'rubber',
  'shape': 'diamond',
  'image': 'img_5.png',
  'image_path': 'clevr_images/unseen_shape/img